In [1]:
import poly_api


markets = [
    "0x0cedbcc216d551bbb704f18cbad8932ea427139760e3dcf715c4a134c875474e",
    "0x15dbb75d18291f1341ef0d33328a3bcc72cc0872b4dd478a9ce9b00b6dd9dd2c",
    "0x2118aa01cc2345896b160331291cf0e1d4146da1d9413f1aa214ffbdd0134937",
    "0x2dded342861dd48b5a239c51a9b7b1cfee20cb1641c875eec3388807cea843b8",
    "0x453157d57aa34741278cb3d6f20d3d01b138395c3ae9eb34e2b97137fe0f2eaa",
    "0x62157086c05e1f5ae7c72ae72c0c1fe62cf4d0b045ceefc18357802bcd005ef8",
    "0x7a18e4d613d7bf6afa06aa9eb1f9287ca81e841d77ae6d5fabdab37f0c0b5d6c",
    "0x9d84821a6c8b45fcd9dad9f50f1b0fc6cb76de7a68d7686bfefba697c32a6375",
    "0xa356cb30609f25eab2219e0aeeb64f9e5de471213427ff911264892a205e1c57",
    "0xad172e84e2a01b30406245a32cade80ad56eb6941ed9e5b22c73b1b206dc7e11",
    "0xc9501eac519c7b631d0425ea093a127f4552ad52b8fdf4e591cea89b31aad981",
    "0xedf6eed432b16b5473929350ee322fed560a4ba4a70785ed06331eac724e7826",
    "0xef3446eac0c8baefadf48c8007429643bdb3d81fbcee4074395600cc40a7c682",
    "0xf9d0a1390e11c9119cf084b3b86bd883052932a951bf933cf23bdd1c0700bebd",
]
markets_bodies = [poly_api.poly_client.get_market(m) for m in markets]

tokens: list[str] = []
for market in markets_bodies:
    for token in market["tokens"]:
        tokens.append(token["token_id"])

In [2]:
from datetime import datetime

from poly_dataset_loader import (
    query_token_timeseries_by_tokens,
    query_asks_and_bids_by_timeseries,
)

ts = query_token_timeseries_by_tokens(tokens=tokens[9:10])
ts = [(item[0], datetime.fromisoformat(item[1].split('.')[0]), item[2]) for item in ts]
ts[0:10]

[(136, datetime.datetime(2025, 4, 18, 10, 14, 1), 12),
 (164, datetime.datetime(2025, 4, 18, 10, 15, 2), 12),
 (192, datetime.datetime(2025, 4, 18, 10, 16, 2), 12),
 (220, datetime.datetime(2025, 4, 18, 10, 17, 2), 12),
 (248, datetime.datetime(2025, 4, 18, 10, 18, 2), 12),
 (276, datetime.datetime(2025, 4, 18, 10, 20, 7), 12),
 (304, datetime.datetime(2025, 4, 18, 10, 20, 13), 12),
 (332, datetime.datetime(2025, 4, 18, 10, 22, 8), 12),
 (360, datetime.datetime(2025, 4, 18, 10, 22, 28), 12),
 (388, datetime.datetime(2025, 4, 18, 10, 24, 8), 12)]

In [3]:
len(ts)

3653

In [4]:
str(min(ts, key=lambda x: x[1])[1]), str(max(ts, key=lambda x: x[1])[1])

('2025-04-18 10:14:01', '2025-04-20 23:27:01')

In [5]:
import sqlite3

import pandas as pd
from poly_dataset_loader import create_tables, get_database_dir


db_dir = get_database_dir()
conn = sqlite3.connect(db_dir / "order_book.db")
cursor = conn.cursor()
create_tables(cursor)

ts = pd.read_sql_query(
    """
    SELECT timeseries_id, timestamp, token_id
    FROM token_timeseries
    """,
    conn,
)
ts['timestamp'] = pd.to_datetime(ts['timestamp'])

asks = pd.read_sql_query(
    "SELECT timeseries_id, MAX(price) as ask FROM asks GROUP BY timeseries_id", conn
)
bids = pd.read_sql_query(
    "SELECT timeseries_id, MIN(price) as bid FROM bids GROUP BY timeseries_id", conn
)

conn.close()

asks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76557 entries, 0 to 76556
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   timeseries_id  76557 non-null  int64  
 1   ask            76557 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 1.2 MB


In [6]:
ts.info(), asks.info(), bids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102408 entries, 0 to 102407
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   timeseries_id  102408 non-null  int64         
 1   timestamp      102408 non-null  datetime64[ns]
 2   token_id       102408 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 2.3 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76557 entries, 0 to 76556
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   timeseries_id  76557 non-null  int64  
 1   ask            76557 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 1.2 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76557 entries, 0 to 76556
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   timeseries_id  76557 non-null  int64  

(None, None, None)

In [7]:
ts_with_huge_delay = ts[ts["timestamp"].dt.second > 10]
ts_with_huge_delay.info()
ts_with_huge_delay.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2160 entries, 14 to 56095
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   timeseries_id  2160 non-null   int64         
 1   timestamp      2160 non-null   datetime64[ns]
 2   token_id       2160 non-null   int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 67.5 KB


,timeseries_id,timestamp,token_id
14,15,2025-04-18 09:19:33.791081,1
15,16,2025-04-18 09:19:33.791081,2
62,63,2025-04-18 09:43:12.286864,1
63,64,2025-04-18 09:43:12.286864,2
292,293,2025-04-18 10:20:13.974474,1


In [8]:
ts_filtered = ts.loc[~ts.index.isin(ts_with_huge_delay.index)].copy()

ts_filtered['timestamp'] = ts_filtered['timestamp'].dt.floor('min')

ts_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100248 entries, 0 to 102407
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   timeseries_id  100248 non-null  int64         
 1   timestamp      100248 non-null  datetime64[ns]
 2   token_id       100248 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 3.1 MB


In [9]:
ts_a = ts_filtered.merge(asks, on='timeseries_id', how='left')
ts_ab = ts_a.merge(bids, on='timeseries_id', how='left')
ts_ab.info()
ts_ab.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100248 entries, 0 to 100247
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   timeseries_id  100248 non-null  int64         
 1   timestamp      100248 non-null  datetime64[ns]
 2   token_id       100248 non-null  int64         
 3   ask            74881 non-null   float64       
 4   bid            74881 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 3.8 MB


,timeseries_id,timestamp,token_id,ask,bid
0,1,2025-04-18 09:12:00,1,0.80,0.01
1,2,2025-04-18 09:12:00,2,0.99,0.20
2,3,2025-04-18 09:13:00,1,0.80,0.01
3,4,2025-04-18 09:13:00,2,0.99,0.20
4,5,2025-04-18 09:14:00,1,0.80,0.01


In [10]:
ts_ab_filtered = ts_ab.dropna()
ts_ab_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70970 entries, 0 to 100247
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   timeseries_id  70970 non-null  int64         
 1   timestamp      70970 non-null  datetime64[ns]
 2   token_id       70970 non-null  int64         
 3   ask            70970 non-null  float64       
 4   bid            70970 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 3.2 MB


In [11]:
import numpy as np


delta = (ts_ab_filtered["ask"] - ts_ab_filtered["bid"])
np.mean(delta), np.std(delta), np.min(delta), np.max(delta)

(np.float64(0.02003883330984924),
 np.float64(0.03967914945818073),
 np.float64(0.003),
 np.float64(0.98))

In [12]:

df_by_token = [
    group.set_index('timestamp')
    for token_id, group in ts_ab_filtered.groupby('token_id')
]

print(f"Created {len(df_by_token)} groups")
df_by_token[0].info()
df_by_token[0].head()

Created 22 groups
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3636 entries, 2025-04-18 09:12:00 to 2025-04-20 23:27:00
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   timeseries_id  3636 non-null   int64  
 1   token_id       3636 non-null   int64  
 2   ask            3636 non-null   float64
 3   bid            3636 non-null   float64
dtypes: float64(2), int64(2)
memory usage: 142.0 KB


,timeseries_id,token_id,ask,bid
timestamp,,,,
2025-04-18 09:12:00,1,1,0.8,0.01
2025-04-18 09:13:00,3,1,0.8,0.01
2025-04-18 09:14:00,5,1,0.8,0.01
2025-04-18 09:15:00,7,1,0.8,0.01
2025-04-18 09:16:00,9,1,0.8,0.01


In [13]:
diff = df_by_token[0].index.diff()

idxs = np.where(diff != pd.Timedelta(minutes=1))[0]
idxs

array([   0,    7,   26,   27,   28,   29,   65,   66,   67,   68,   69,
        114,  115,  116,  117,  118,  122,  123,  125,  126,  127,  128,
        129,  130,  131,  132,  133,  134,  135,  136,  137,  138,  139,
        140,  141,  142,  143,  144,  145,  146,  147,  148,  149,  150,
        151,  152,  153,  154,  155,  156,  157,  158,  159,  160,  161,
        162,  163,  164,  165,  166,  167,  168,  169, 1425, 1427, 1428,
       1429, 1430, 1431, 1432, 1979, 1980, 1981, 1982, 1983])

In [14]:

diff = 0
for i in range(len(idxs) - 1):
    diff = max(diff, idxs[i+1] - idxs[i])

diff

np.int64(1256)

In [15]:
subset = df_by_token[0].iloc[169:1425]
subset

,timeseries_id,token_id,ask,bid
timestamp,,,,
2025-04-18 13:13:00,4717,1,0.16,0.11
2025-04-18 13:14:00,4745,1,0.16,0.11
2025-04-18 13:15:00,4773,1,0.16,0.11
2025-04-18 13:16:00,4801,1,0.16,0.11
2025-04-18 13:17:00,4829,1,0.16,0.11
...,...,...,...,...
2025-04-19 10:04:00,39745,1,0.15,0.10
2025-04-19 10:05:00,39773,1,0.15,0.10
2025-04-19 10:06:00,39801,1,0.15,0.10


In [16]:
np.mean(subset["ask"] - subset["bid"])

np.float64(0.05196656050955415)

In [17]:
def trading_func_use_capped(
    idx: int,
    shares: np.ndarray,
    binance_delta: np.ndarray,
    traded: np.ndarray,
    poly_bid: np.ndarray,
    poly_ask: np.ndarray,
    balance: np.ndarray,
    min_delta: float,
    transaction_fee: float,
    purchase_cap: float = 100,
):
    btc_price_grew = binance_delta[idx] > min_delta
    holding_shares = shares[idx - 1] > 0

    if not holding_shares:
        if btc_price_grew:
            # Buy shares
            spend = min(purchase_cap + transaction_fee, balance[idx - 1])
            spend_on_shares = spend - transaction_fee
            shares[idx] = spend_on_shares / poly_ask[idx]
            balance[idx] = balance[idx - 1] - spend
            traded[idx] = True

    elif holding_shares:
        # Sell shares
        balance[idx] = (
            balance[idx - 1]
            + (shares[idx - 1] * poly_bid[idx])
            - transaction_fee
        )
        shares[idx] = 0
        traded[idx] = True

    # If a new balance was not set, keep the previous one
    if not traded[idx]:
        balance[idx] = balance[idx - 1]
        shares[idx] = shares[idx - 1]


def simulate_trading(
    poly_bid: np.ndarray,
    poly_ask: np.ndarray,
    binance_delta: pd.Series,
    starting_balance=100,
    min_delta=30,
    transaction_fee=0.0,
):
    # Add an extra item at the beginning to simplify indexing
    poly_bid = np.concat((np.array([poly_bid[0]]), poly_bid))
    poly_ask = np.concat((np.array([poly_ask[0]]), poly_ask))

    binance_delta = np.concat(([0], binance_delta.to_numpy()))

    # TODO: My trades can influence the market. Add some cap to the amount of shares I can buy/sell?
    # TODO: Not all my shares can get bought. Account for that too.
    # TODO: Does it make sense to look at consecutive change of BTC price?
    #       Datapoints where BTC grew 2+ consecutive intervals at a time?
    balance = np.zeros(len(poly_bid)) * np.nan
    shares = np.zeros(len(poly_bid)) * np.nan
    traded = np.zeros(len(poly_bid))

    balance[0] = starting_balance
    shares[0] = 0

    idx = 1
    while idx < (len(poly_bid) - 1):
        trading_func_use_capped(
            idx,
            shares,
            binance_delta,
            traded,
            poly_bid,
            poly_ask,
            balance,
            min_delta,
            transaction_fee,
        )
        idx += 1

    # Set the last balance and shares to some value (NaN by default)
    balance[idx] = balance[idx - 1]

    # If has shares at the end, sell them all
    if shares[idx] > 0:
        aquired_balance = shares[idx] * poly_ask[idx - 1]
        balance[idx] = balance[idx - 1] + aquired_balance - transaction_fee
        shares[idx] = 0
        traded[idx] = True
    else:
        shares[idx] = shares[idx - 1]

    actives = balance + (shares * poly_ask)
    actives_end = actives[1:]
    actives_start = actives[:-1]
    actives_delta = actives_end - actives_start
    actives_delta_percentage = actives_delta / actives_start

    trades_made = np.sum(traded)
    total_fees = transaction_fee * trades_made
    return {
        "actives_start": actives[0],
        "actives_end": round(float(actives[idx]), 4),
        "actives": actives,
        "actives_delta": actives_delta,
        "actives_delta_percentage": actives_delta_percentage,
        "actives_won_lost": (
            round(float(sum(actives_delta[actives_delta > 0])), 4),
            round(float(sum(actives_delta[actives_delta < 0])), 4),
        ),
        "trades_made": trades_made,
        "trades_won_lost": (
            int(sum(actives_delta > 0)),
            int(sum(actives_delta < 0)),
        ),
        "trade_fees": round(total_fees, 4),
        "trade_descisions_made": len(poly_bid),
    }


def print_result(result: dict):
    for key, value in result.items():
        if not isinstance(value, np.ndarray):
            print(f"{key}: {value}")

In [26]:
from binance.client import Client

import binance_datasets
from poly_datasets import PMDataset

binance_df = binance_datasets.load_binance_dataset(
    "BTCUSDT", 
    1,
    start=min(df_by_token[0].index),
    end=max(df_by_token[0].index),
)
binance_df.info()

r = simulate_trading(
    poly_bid=df_by_token[0]["bid"].to_numpy(),
    poly_ask=df_by_token[0]["ask"].to_numpy(),
    binance_delta=binance_df["delta"],
    starting_balance=100,
    min_delta=100,
    transaction_fee=0.0,
)

print_result(r)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3736 entries, 2025-04-18 09:12:00 to 2025-04-20 23:27:00
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Open                          3736 non-null   float64       
 1   High                          3736 non-null   float64       
 2   Low                           3736 non-null   float64       
 3   Close                         3736 non-null   float64       
 4   Volume                        3736 non-null   float64       
 5   Close Time                    3736 non-null   datetime64[ns]
 6   Quote Asset Volume            3736 non-null   float64       
 7   Number of Trades              3736 non-null   int64         
 8   Taker Buy Base Asset Volume   3736 non-null   float64       
 9   Taker Buy Quote Asset Volume  3736 non-null   float64       
 10  Ignore                        3736 non-null   object        